In [3]:
import os
import json
import time
import chess
import chess
import chess.engine
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import json
from chess_analysis import extract_score, evaluate_moves, decode, process_game

In [4]:
%%file chess_analysis.py
import os
import json
import time
import chess
import chess.engine
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import json

engine_path = "/opt/homebrew/bin/stockfish"


def extract_score(score_obj):
    if score_obj.is_mate():
        # Return 99 or -99 depending on the sign of the mate count
        return 99 if score_obj.white().mate() > 0 else -99
    else:
        # Convert centipawn score to regular pawn units
        return score_obj.white().score() / 100

def evaluate_moves(moves, engine_path, multi_pv_lines=5, thinking_time=1):
    try:
        board = chess.Board()
        engine = chess.engine.SimpleEngine.popen_uci(engine_path)

        evaluations = []

        for move in tqdm(moves):
            # Construct the UCI string, considering pawn promotions
            uci_move = move['from'] + move['to']
            if 'promotion' in move:
                uci_move += move['promotion'].lower()

            # Find the number of legal moves in the position
            legal_moves_count = len(list(board.legal_moves))

            # Analyse the position to the desired depth with multi-PV
            multi_pv_result = engine.analyse(board, chess.engine.Limit(time=thinking_time), multipv=min(multi_pv_lines, legal_moves_count))

            # Extract the moves and evaluations from the engine's output
            pv_moves = [info.get('pv')[0] for info in multi_pv_result if info.get('pv')]
            pv_evals = [extract_score(info.get('score')) for info in multi_pv_result]

            # If the actual move is in the top multi-PV lines, get its rank and eval, otherwise set them to -1
            actual_move = board.push_uci(uci_move)

            if actual_move in pv_moves:
                rank = pv_moves.index(actual_move) + 1
                actual_eval = pv_evals[pv_moves.index(actual_move)]
            else:
                rank = -1
                actual = engine.analyse(board, chess.engine.Limit(time=thinking_time))
                actual_eval = extract_score(actual['score'])

            best_move = pv_moves[0]
            best_eval = pv_evals[0]

            evaluations.append({
                'Best Move': best_move,
                'Best Move Eval': best_eval,
                'Ranking Real Move': rank,
                'Real Move Eval': actual_eval
            })

        engine.quit()
        return evaluations
    except Exception as e:
        print(e)
        engine.quit()
        return None


T = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!?{~}(^)[_]@#$,./&-*++="

def decode(e):
    f = []
    g = len(e)
    for c in range(0, g, 2):
        d = {}
        b = T.index(e[c])
        a = T.index(e[c + 1])
        if a > 63:
            d["promotion"] = "qnrbkp"[int((a - 64) / 3)]
            a = b + (-8 if b < 16 else 8) + (a - 1) % 3 - 1
        if b > 75:
            d["drop"] = "qnrbkp"[b - 79]
        else:
            d["from"] = T[b % 8] + str(int(b / 8) + 1)
        d["to"] = T[a % 8] + str(int(a / 8) + 1)
        f.append(d)
    return f

def process_game(game, engine_path, multi_pv_lines=5, thinking_time=1):
    new_filename = game[:-5] + "_analysed.json"  # Assuming '.json' extension for the original game files
    if os.path.exists("../Data/Analysed/" + new_filename):
        return
    # Load game data
    with open("../Data/Games/" + game) as f:
        game_json = json.load(f)

    enc_movelist = game_json["game"]["moveList"]
    movelist = decode(enc_movelist)
    evaluation = evaluate_moves(movelist, engine_path, multi_pv_lines=multi_pv_lines, thinking_time=thinking_time)
    if not evaluation:
        print(f"Error evaluating game {game}. Skipping...")
        return
    # Modify the evaluation dictionaries
    for index, eval_dict in enumerate(evaluation):
        eval_dict["Best Move"] = eval_dict["Best Move"].uci()  # Convert chess.Move to string
        eval_dict["plycount"] = index + 1
        eval_dict["player"] = "white" if eval_dict["plycount"] % 2 == 1 else "black"
        eval_dict["difference"] = eval_dict["Real Move Eval"] - eval_dict["Best Move Eval"]
        eval_dict["difference"] *= -1 if eval_dict["player"] == "white" else 1

    # Append evaluations and additional metadata to the game's JSON
    game_json["evaluations"] = evaluation
    game_json["multi_pv_lines"] = multi_pv_lines
    game_json["thinking_time"] = thinking_time
    game_json["timestamp"] = int(time.time())

    # Write to a new file
    with open("../Data/Analysed/" + new_filename, "w") as f:
        json.dump(game_json, f, indent=4)

def process_game_helper(args):
    process_game(*args)

Overwriting chess_analysis.py


In [5]:
from multiprocessing import Pool
from chess_analysis import process_game_helper
import os 
import time 

def main():
    engine_path = "/opt/homebrew/bin/stockfish"
    #games = os.listdir("../Data/Games/")
    games = [x for x in os.listdir("../Data/Games/") if "2023" in x]
    # Using 8 CPUs
    num_processes = 10

    start_time = time.time()

    with Pool(processes=num_processes) as pool:
        pool.map(process_game_helper, [(game, "/opt/homebrew/bin/stockfish") for game in games])

    end_time = time.time()
    print(f"Finished processing in {end_time - start_time} seconds.")

if __name__ == "__main__":
    main()


0it [00:00, ?it/s]/138 [00:00<?, ?it/s]
0it [00:00, ?it/s]/58 [00:00<?, ?it/s]
  0%|          | 0/126 [00:00<?, ?it/s]

Error evaluating game late-titled-tuesday-blitz-february-07-2023-3796152_69552435777.json. Skipping...
Error evaluating game early-titled-tuesday-blitz-march-21-2023-3920018_73157041787.json. Skipping...


0it [00:00, ?it/s]
0it [00:00, ?it/s]


Error evaluating game early-titled-tuesday-blitz-may-02-2023-4002919_76782248267.json. Skipping...
Error evaluating game late-titled-tuesday-blitz-april-11-2023-3961659_74987644705.json. Skipping...


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]/126 [00:01<02:05,  1.00s/it]


...

0it [00:00, ?it/s]5/120 [01:18<01:05,  1.19s/it]
  0%|          | 0/115 [00:00<?, ?it/s]

Error evaluating game late-titled-tuesday-blitz-april-11-2023-3961659_74987028663.json. Skipping...


100%|██████████| 132/132 [02:30<00:00,  1.14s/it]
0it [00:00, ?it/s]03/144 [02:08<00:52,  1.29s/it]
 77%|███████▋  | 59/77 [01:12<00:27,  1.53s/it]

Error evaluating game early-titled-tuesday-blitz-april-11-2023-3961658_74965495445.json. Skipping...


 57%|█████▋    | 91/159 [01:40<01:08,  1.01s/it]